# Print the Branch Contents of ROOT files created for the Machine Learning Pipeline
---

This notebook uses `uproot` to load a single event from a given root file and displays the branches with their corresponding python and c++ data types and the values for a given number of lines.

The structure of the root file, meaning its trees and branches, are hard coded. If the branch or tree structure gets edited, so must this code be changed.

## Imports

In [6]:
import uproot as up
import pandas as pd

## Load the ROOT File and Display its Branches

In [2]:
# Path to the ROOT file
mlDataFilePath = "/home/nikin105/mlProject/data/machineLearning/mum5mup5/root/mlData.root" # <-- Change this to the path of the ROOT file

hits = up.open(f"{mlDataFilePath}:hits")
cells = up.open(f"{mlDataFilePath}:cells")
particles = up.open(f"{mlDataFilePath}:particles")
truth = up.open(f"{mlDataFilePath}:truth")

print(hits.show())
print(cells.show())
print(particles.show())
print(truth.show())

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
hit_id               | RVec<int32_t>            | AsJagged(AsDtype('>i4'), he...
x                    | RVec<double>             | AsJagged(AsDtype('>f8'), he...
y                    | RVec<double>             | AsJagged(AsDtype('>f8'), he...
z                    | RVec<double>             | AsJagged(AsDtype('>f8'), he...
volume_id            | RVec<int32_t>            | AsJagged(AsDtype('>i4'), he...
layer_id             | RVec<int32_t>            | AsJagged(AsDtype('>i4'), he...
module_id            | RVec<int32_t>            | AsJagged(AsDtype('>i4'), he...
None
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
hit_id               | RVec<int32_t>            | AsJagged(AsDtype('>i4'), he...
dep_charge           | 

In [3]:
# Event that should be displayed
event = 0 # <-- Change this to the event number that should be displayed

hitBranchNames = ["hit_id", "x", "y", "z", "volume_id", "layer_id", "module_id"]
hitParams = hits.arrays(hitBranchNames, library="np", entry_start=event, entry_stop=event+1)

cellBranchNames = ["hit_id", "dep_charge", "energy_loss", "volume_id", "layer_id", "module_id", "sector_id", "isochrone", "skewed"]
cellParams = cells.arrays(cellBranchNames, library="np", entry_start=event, entry_stop=event+1)

truthBranchNames = ["hit_id", "tx", "ty", "tz", "tpx", "tpy", "tpz", "particle_id"]
truthParams = truth.arrays(truthBranchNames, library="np", entry_start=event, entry_stop=event+1)

particlesBranchNames = ["particle_id", "vx", "vy", "vz", "px", "py", "pz", "nhits", "pdgcode", "start_time", "primary"]
particlesParams = particles.arrays(particlesBranchNames, library="np", entry_start=event, entry_stop=event+1)

In [27]:
def save_tree_entry_as_pandas(root_file : up.ReadOnlyDirectory, tree_name: str, event_num: int
    ) -> pd.DataFrame:
        """
        Extracts all data from all TBranches in a TTree for a single event and saves it in a pandas DataFrame.

        This function expects that the TBranches contain a vector for each event, while the vector of each
        branch / column has the same length.

        Args:
            tree_name (str): Name of the TTree to read.
            event_num (int): Number of the event to read. Corresponds to the entry number in the ROOT file.

        Returns:
            pd.DataFrame: Pandas DataFrame containing the data from the TTree for the specified event.
        """

        # Get the names of all branches in the TTree
        branches = root_file[tree_name].keys()
        
        # Prepare the dictionary to store the branch data in
        tree_data = {}

        # Read the data from each branch and store it in the dictionary
        for branch in branches:
            branchData = root_file[tree_name + "/" + branch].array(
                entry_start=event_num, entry_stop=event_num + 1
            )
            # Save the contents of the branch in the dictionary
            tree_data[branch] = branchData[0]

            # Return the dictionary as a pandas DataFrame
        return pd.DataFrame(tree_data)

In [39]:
rootFile = up.open(mlDataFilePath)

part = save_tree_entry_as_pandas(rootFile, "particles", 1)
tru = save_tree_entry_as_pandas(rootFile, "truth", 1)

print(tru.head(n=10))

tru = tru.merge(
            part[["particle_id", "px", "py", "pz", "pdgcode"]],
            on="particle_id",
            how="left",
        )

print(part.head(n=10))
print(tru.head(n=10))


   hit_id        tx         ty         tz        tT       tpx       tpy  \
0       0  4.840407  16.298269  28.123190  1.083077  0.228931  0.674997   
1       1  5.150823  17.203552  29.697575  1.129586  0.232457  0.674102   
2       2  5.463572  18.104769  31.266464  1.193881  0.235143  0.673330   
3       3  5.717229  18.828005  32.526299  1.255448  0.237097  0.672992   
4       4  6.036303  19.725880  34.091915  1.301550  0.239905  0.671991   
5       5  6.357416  20.620289  35.653183  1.366444  0.242228  0.671219   
6       6  6.620961  21.347149  36.923286  1.424829  0.244056  0.670534   
7       7  6.947878  22.238281  38.482967  1.473155  0.246622  0.669483   
8       8  7.281619  23.139122  40.060772  1.541255  0.249190  0.669089   
9       9  7.617163  24.033981  41.629990  1.596822  0.251707  0.667762   

        tpz  particle_id  
0  1.172967            9  
1  1.172720            9  
2  1.172596            9  
3  1.172362            9  
4  1.172318            9  
5  1.172243 

In [5]:
# Number of lines that should be printed
lines = 11 # <-- Change this to the number of lines that should be printed

print("hit_id,x,y,z,volume_id,layer_id,module_id")
for i in range(lines):
    print(hitParams["hit_id"][0][i],hitParams["x"][0][i],hitParams["y"][0][i],hitParams["z"][0][i],hitParams["volume_id"][0][i],hitParams["layer_id"][0][i],hitParams["module_id"][0][i])
    
print("\nhit_id", "dep_charge", "energy_loss", "volume_id", "layer_id", "module_id", "sector_id", "isochrone", "skewed")
for i in range(lines):
    print(cellParams["hit_id"][0][i],cellParams["dep_charge"][0][i],cellParams["energy_loss"][0][i],cellParams["volume_id"][0][i],cellParams["layer_id"][0][i],cellParams["module_id"][0][i], cellParams["sector_id"][0][i], cellParams["isochrone"][0][i], cellParams["skewed"][0][i])
    
print("\nhit_id", "tx", "ty", "tz", "tpx", "tpy", "tpz", "particle_id")
for i in range(lines):
    print(truthParams["hit_id"][0][i],truthParams["tx"][0][i],truthParams["ty"][0][i],truthParams["tz"][0][i],truthParams["tpx"][0][i],truthParams["tpy"][0][i],truthParams["tpz"][0][i],truthParams["particle_id"][0][i])
    
print("\nparticle_id", "vx", "vy", "vz", "px", "py", "pz", "nhits", "pdgcode", "start_time", "primary")
for i in range(lines):
    print(particlesParams["particle_id"][0][i],particlesParams["vx"][0][i],particlesParams["vy"][0][i],particlesParams["vz"][0][i],particlesParams["px"][0][i],particlesParams["py"][0][i],particlesParams["pz"][0][i],particlesParams["nhits"][0][i],particlesParams["pdgcode"][0][i],particlesParams["start_time"][0][i],particlesParams["primary"][0][i])

hit_id,x,y,z,volume_id,layer_id,module_id
0 13.120285034179688 -11.614999771118164 35.0 9 0 65
1 13.994970321655273 -12.119999885559082 35.0 9 1 173
2 13.994970321655273 -13.130000114440918 35.0 9 2 286
3 14.869656562805176 -13.635000228881836 35.0 9 3 406
4 15.744341850280762 -14.140000343322754 35.0 9 4 532
5 16.619028091430664 -14.645000457763672 35.0 9 5 664
6 16.619028091430664 -15.654999732971191 35.0 9 6 801
7 17.49371337890625 -16.15999984741211 35.0 9 7 945
8 15.891955375671387 -18.251022338867188 35.0 9 8 1109
9 16.767242431640625 -18.75567626953125 35.0 9 9 1307
10 22.252803802490234 -17.036579132080078 26.463886260986328 9 10 1517

hit_id dep_charge energy_loss volume_id layer_id module_id sector_id isochrone skewed
0 4589899.714998919 4.589899714998919 9 0 65 3 0.04168410398405612 0
1 22502325.37145041 22.502325371450407 9 1 173 3 0.2944226189791269 0
2 3136079.9820130877 3.136079982013088 9 2 286 3 0.46971404277654916 0
3 7687911.673469984 7.6879116734699835 9 3 406 3 0.2